<a href="https://colab.research.google.com/github/HFM007/Analisis-Sentimen-dengan-IndoBERT-dan-IndoRoberta/blob/main/AutomatedLabelingWithIndoBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, IntervalStrategy
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import numpy as np
import os
from datasets import Dataset, Value

# Set WANDB_DISABLED to avoid issues if Weights & Biases is not configured
os.environ["WANDB_DISABLED"] = "true"

# Langkah 1: Muat Dataset Berlabel dan pra-pemrosesan
# Membaca file CSV 'ada labelnya - Sheet1.csv' yang berisi data komentar TikTok dan label sentimen.
df_labeled = pd.read_csv("label_manual - Sheet1.csv") #

# Mengganti nama kolom untuk konsistensi: 'Data Komentar TIKTOK' menjadi 'text' dan 'Labeling' menjadi 'label'.
df_labeled.rename(columns={'komentar': 'text', 'Labeling': 'label'}, inplace=True) 

# Menghapus baris yang memiliki nilai kosong di kolom 'text' atau 'label'.
df_labeled = df_labeled.dropna(subset=['text', 'label'])

# Membuat peta label untuk mengkonversi label teks menjadi numerik: netral=0, positif=1, negatif=2.
label_map = {'Netral': 0, 'Positif': 1, 'Negatif': 2}
df_labeled['label'] = df_labeled['label'].map(label_map)

# Langkah 2: Inisialisasi Model dan Tokenizer IndoBERT
# Menentukan nama model IndoBERT yang akan digunakan dari Hugging Face.
model_name = "indobenchmark/indobert-base-p1"

# Menginisialisasi tokenizer berdasarkan model yang dipilih.
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Menginisialisasi model klasifikasi sekuensial dengan jumlah label yang sesuai.
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_map))

# Langkah 3: Tokenisasi Data Berlabel
# Mendefinisikan fungsi untuk melakukan tokenisasi pada teks.
def tokenize_function(examples):
  # Mengkonversi semua teks ke string, menangani nilai None.
  texts = [str(text) if text is not None else "" for text in examples['text']]
  # Melakukan tokenisasi dengan pemotongan (truncation) dan padding agar panjangnya seragam.
  return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

# Mengkonversi DataFrame berlabel menjadi objek Dataset.
dataset_labeled = Dataset.from_pandas(df_labeled)

# Menerapkan fungsi tokenisasi pada dataset.
dataset_labeled = dataset_labeled.map(tokenize_function, batched=True)

# Langkah 4: Split data berlabel menjadi set pelatihan dan pengujian
# Membagi dataset berlabel menjadi 80% untuk pelatihan dan 20% untuk pengujian.
# 'seed=42' digunakan untuk memastikan pembagian yang konsisten setiap kali kode dijalankan.
dataset_labeled = dataset_labeled.train_test_split(test_size=0.2, seed=42)

train_dataset = dataset_labeled["train"]
test_dataset = dataset_labeled["test"]

# Set format of label column to integer to prevent tensor conversion errors
train_dataset = train_dataset.cast_column("label", Value("int64"))
test_dataset = test_dataset.cast_column("label", Value("int64"))

# Add filter to ensure labels are integers
train_dataset = train_dataset.filter(lambda x: isinstance(x['label'], int) and x['label'] in label_map.values())
test_dataset = test_dataset.filter(lambda x: isinstance(x['label'], int) and x['label'] in label_map.values())


# Langkah 5: Set training config & trainer
# Mengatur argumen pelatihan untuk Trainer Transformers.
training_args = TrainingArguments(
    output_dir="./results_indobert", # Direktori untuk menyimpan hasil pelatihan model.
    eval_strategy=IntervalStrategy.EPOCH, # Strategi evaluasi dilakukan per epoch.
    per_device_train_batch_size=8, # Ukuran batch per perangkat (GPU/CPU) untuk pelatihan.
    per_device_eval_batch_size=8, # Ukuran batch per perangkat untuk evaluasi.
    num_train_epochs=3, # Jumlah epoch pelatihan.
    logging_dir="./logs_indobert", # Direktori untuk menyimpan log pelatihan.
    logging_steps=10, # Log setiap 10 langkah pelatihan.
    save_strategy=IntervalStrategy.EPOCH, # Menyimpan checkpoint model setiap epoch.
    load_best_model_at_end=True, # Memuat model terbaik (berdasarkan metrik) setelah pelatihan selesai.
    metric_for_best_model="accuracy", # Metrik yang digunakan untuk menentukan model terbaik.
    greater_is_better=True # Akurasi yang lebih tinggi dianggap lebih baik.
)

# Mendefinisikan fungsi compute_metrics untuk menghitung metrik evaluasi (akurasi, presisi, recall, F1-score).
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1) # Mengambil indeks kelas dengan probabilitas tertinggi.
    accuracy = accuracy_score(labels, predictions)
    # Menghitung presisi untuk rata-rata 'macro' dan 'weighted'. 'zero_division=0' untuk menangani kasus tanpa prediksi.
    precision_macro = precision_score(labels, predictions, average='macro', zero_division=0)
    precision_weighted = precision_score(labels, predictions, average='weighted', zero_division=0)
    # Menghitung recall untuk rata-rata 'macro' dan 'weighted'.
    recall_macro = recall_score(labels, predictions, average='macro', zero_division=0)
    recall_weighted = recall_score(labels, predictions, average='weighted', zero_division=0)
    # Menghitung F1-score untuk rata-rata 'macro' dan 'weighted'.
    f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)
    f1_weighted = f1_score(labels, predictions, average='weighted', zero_division=0)
    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_weighted': recall_weighted,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
    }

# Menginisialisasi objek Trainer dengan model, argumen pelatihan, dataset, tokenizer, dan fungsi metrik.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Langkah 6: Memulai training model
print("Memulai training model IndoBERT...")
trainer.train()

# Langkah 7: Evaluasi Model pada test_dataset
print("\nEvaluasi model IndoBERT pada test dataset:")
results_eval = trainer.evaluate()
print(results_eval)

# Menentukan ambang batas akurasi dan presisi untuk keandalan model.
accuracy_threshold = 0.85
precision_threshold = 0.85

# Mengambil nilai akurasi dan presisi dari hasil evaluasi.
accuracy = results_eval.get('eval_accuracy', 0)
precision_macro = results_eval.get('eval_precision_macro', 0)
precision_weighted = results_eval.get('eval_precision_weighted', 0)

print(f"\nAccuracy pada test set: {accuracy:.4f}")
print(f"Precision (macro) pada test set: {precision_macro:.4f}")
print(f"Precision (weighted) pada test set: {precision_weighted:.4f}")

# Memeriksa apakah model memenuhi kriteria keandalan.
if accuracy >= accuracy_threshold and precision_macro >= precision_threshold and precision_weighted >= precision_threshold:
    print(f"\nHasil evaluasi model IndoBERT pada test set memenuhi kriteria (Accuracy >= {accuracy_threshold*100}%, Precision (macro) >= {precision_threshold*100}%, Precision (weighted) >= {precision_threshold*100}%).")
    print("Model dianggap andal untuk automated labeling.")

    # Langkah 8: Lakukan Automated Labeling pada Data Tanpa Label
    # Membaca file CSV 'gaada labelnya - Sheet1.csv' yang akan di-label secara otomatis.
    df_no_label = pd.read_csv("belum_dilabeli - Sheet1.csv") #
    # Mengganti nama kolom untuk konsistensi.
    df_no_label.rename(columns={'komentar': 'text'}, inplace=True) #
    # Menghapus baris yang memiliki nilai kosong di kolom 'text'.
    df_no_label = df_no_label.dropna(subset=['text'])

    # Mengkonversi DataFrame tanpa label menjadi objek Dataset dan melakukan tokenisasi.
    dataset_no_label = Dataset.from_pandas(df_no_label)
    dataset_no_label = dataset_no_label.map(tokenize_function, batched=True)
    # Mengatur format dataset ke PyTorch tensor untuk prediksi.
    dataset_no_label.set_format(type="torch", columns=["input_ids", "attention_mask"])

    print("\nMelakukan prediksi pada data tanpa label menggunakan model IndoBERT yang terlatih...")
    # Melakukan prediksi sentimen pada dataset tanpa label menggunakan model yang telah dilatih.
    predictions_no_label = trainer.predict(dataset_no_label)
    # Mengambil label yang diprediksi (kelas dengan probabilitas tertinggi).
    predicted_labels_no_label = predictions_no_label.predictions.argmax(-1)

    # Memetakan kembali label numerik yang diprediksi ke label teks.
    reverse_label_map = {v: k for k, v in label_map.items()}
    df_no_label['predicted_label'] = predicted_labels_no_label
    df_no_label['predicted_label_text'] = df_no_label['predicted_label'].map(reverse_label_map)

    # Langkah 9: Simpan hasil automated labeling ke file Excel baru
    output_file_name = "automated_sentiment_indobert_results.xlsx"
    # Menyimpan hasil labeling otomatis ke sheet 'Automated_Labels' dan metrik evaluasi model ke sheet 'Model_Evaluation_Metrics'.
    with pd.ExcelWriter(output_file_name) as writer:
        df_no_label.to_excel(writer, sheet_name="Automated_Labels", index=False)
        metrics_df = pd.DataFrame([results_eval])
        metrics_df.to_excel(writer, sheet_name="Model_Evaluation_Metrics", index=False)

    print(f"\nHasil automated labeling dan metrik evaluasi model disimpan ke {output_file_name}")

else:
    print(f"\nHasil evaluasi model IndoBERT pada test set TIDAK memenuhi kriteria (Accuracy >= {accuracy_threshold*100}%, Precision (macro) >= {precision_threshold*100}%, Precision (weighted) >= {precision_threshold*100}%).")
    print("Model belum dianggap andal untuk automated labeling.")
    print("Automated labeling tidak dilakukan karena model belum memenuhi kriteria keandalan.")